In [ ]:
import numpy as np
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, and_

from flask import Flask, jsonify

################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Measurement = Base.classes.measurement
Station = Base.classes.station

#################################################
# Flask Setup
#################################################
app = Flask(__name__)

#################################################
# Flask Routes
#################################################
@app.route("/")
def home():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/&lt;start&gt;<br/>"
        f"/api/v1.0/&lt;start&gt;/&lt;end&gt;"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)
    sel = [Measurement.date,Measurement.prcp]
    queryresult = session.query(*sel).all()
    session.close()

    precipitation = []
    for date, prcp in queryresult:
        prcp_dict = {}
        prcp_dict["Date"] = date
        prcp_dict["Precipitation"] = prcp
        precipitation.append(prcp_dict)

    return jsonify(precipitation)

@app.route("/api/v1.0/stations")
def stations():
    def stations():
    session = Session(engine)
    sel = [Station.station,Station.name,Station.latitude,Station.longitude,Station.elevation]
    queryresult = session.query(*sel).all()
    session.close()

    stations = []
    for station,name,lat,lon,el in queryresult:
        station_dict = {}
        station_dict["Station"] = station
        station_dict["Name"] = name
        station_dict["Lat"] = lat
        station_dict["Lon"] = lon
        station_dict["Elevation"] = el
        stations.append(station_dict)

    return jsonify(stations)

@app.route("/api/v1.0/tobs")
def TOBS():
    session = Session(engine)
    lateststr = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    latestdate = dt.datetime.strptime(lateststr, '%Y-%m-%d')
    querydate = dt.date(latestdate.year -1, latestdate.month, latestdate.day)
    sel = [Measurement.date,Measurement.TOBS]
    queryresult = session.query(*sel).filter(Measurement.date >= querydate).all()
    session.close()

    TOBS_list = []
    for date, TOBS_list in queryresult:
        TOBS_dict = {}
        TOBS_dict["Date"] = date
        TOBS_dict["Tobs"] = tobs
        TOBS_list.append(TOBS_dict)

    return jsonify(TOBS_list)

@app.route("/api/v1.0/<start>")
def temp_range_start(start):
    session = Session(engine)
    results = session.query(Measurement.date,func.min(Measurement.TOBS),func.avg(Measurement.TOBS),func.max(Measurement.TOBS)).filter(Measurement.date >= start).group_by(Measurement.date).all()

    return_start_list = []
    for date, min, avg, max in results:
        new_dict = {}
        new_dict["Date"] = date
        new_dict["TMIN"] = min
        new_dict["TAVG"] = avg
        new_dict["TMAX"] = max
        return_start_list.append(new_dict)

    session.close()    

    return jsonify(return_start_list)

@app.route("/api/v1.0/<start>/<end>")
def temp_range_start_end(start,end):
    session = Session(engine)
    results = session.query(Measurement.date,func.min(Measurement.TOBS),func.avg(Measurement.TOBS),func.max(Measurement.TOBS)).filter(and_(Measurement.date >= start, Measurement.date <= end)).group_by(Measurement.date).all()

    return_end_list = []
    for date, min, avg, max in results:
        new_dict = {}
        new_dict["Date"] = date
        new_dict["TMIN"] = min
        new_dict["TAVG"] = avg
        new_dict["TMAX"] = max
        return_end_list.append(new_dict)

    session.close()    

    return jsonify(return_end_list)

if __name__ == '__main__':
    app.run(debug=True)